In [75]:
import pandas as pd
import re

In [76]:
from tabulate import tabulate

## Codigo de Entrada

In [77]:
nome_arquivo = "input.p"
try:
    codigo_entrada = open("lib/input/" + nome_arquivo, "r", encoding="utf8").read()
except:
    codigo_entrada = """"""

In [78]:
print(codigo_entrada)

begin end if ; : :=
while for then 20 5.5 x1 a program


## Definição de DataFrame resultante

In [79]:
rows = []
df = pd.DataFrame(rows, columns=["Tipo", "Palavra", "Linha", "Posicao_Inicio", "Posicao_Fim"])
df

,Tipo,Palavra,Linha,Posicao_Inicio,Posicao_Fim


In [80]:
def procurar_palavras(rows, tipo, lista_palavras = ["entrada"]):
    for idx, palavra in enumerate(lista_palavras):
        if tipo in ["operadores","pontuação"]:
            regex = rf"(?<!:){re.escape(palavra)}(?!=)"
        elif tipo in ["inteiro"]:
            regex = r"[-+]?(?<!\w)(?<![\d.])\d+(?![\d.])"
        elif tipo in ["real"]:
            regex = r"[-+]?[0-9]*(\.[0-9]+)"
        else:
            regex = rf"(?i)\b{re.escape(palavra)}\b"

        matches = re.finditer(regex, codigo_entrada, re.MULTILINE)

        for matchNum, match in enumerate(matches, start=1):
            
            line = codigo_entrada[0:match.end()].count('\n') + 1   
            rows.append([tipo, match.group(), line, match.start(), match.end()])
    
    return rows

## Procurar Palavras Reservadas no Código

In [81]:
tipo = "palavra reservada"
try:
    palavra_procurada = pd.read_csv("lib/model/palavras_reservadas.csv", header=None)
    palavra_procurada = list(palavra_procurada[0])
except:
    palavra_procurada = ["begin", "end", "if", "while", "for", "then", "program"]

In [82]:
rows = procurar_palavras(rows, tipo, palavra_procurada)

## Procurar Variáveis no Código

In [83]:
tipo = "variável"
try:
    palavra_procurada = pd.read_csv("lib/model/variaveis.csv", header=None)
    palavra_procurada = list(palavra_procurada[0])
except:
    palavra_procurada = ["a", "x1"]

In [84]:
rows = procurar_palavras(rows, tipo, palavra_procurada)

## Procurar Operadores no Código

In [85]:
tipo = "operadores"
try:
    palavra_procurada = pd.read_csv("lib/model/operadores.csv", header=None)
    palavra_procurada = list(palavra_procurada[0])
except:
    palavra_procurada = [":="]

In [86]:
rows = procurar_palavras(rows, tipo, palavra_procurada)

## Procurar Pontuação no Código

In [87]:
tipo = "pontuação"
try:
    palavra_procurada = pd.read_csv("lib/model/pontuacoes.csv", header=None)
    palavra_procurada = list(palavra_procurada[0])
except:
    palavra_procurada = [";", ":"]

In [88]:
rows = procurar_palavras(rows, tipo, palavra_procurada)

## Procurar Números Inteiros no Código

In [89]:
tipo = "inteiro"

In [90]:
rows = procurar_palavras(rows, tipo)

## Procurar Números Decimais no Código

In [91]:
tipo = "real"

In [92]:
rows = procurar_palavras(rows, tipo)

## DataFrame resultante

In [93]:
df = pd.DataFrame(rows, columns=["Tipo", "Palavra", "Linha", "Posicao_Inicio", "Posicao_Fim"])

In [94]:
df = df.sort_values(by=['Posicao_Inicio']).reset_index(drop=True)

## Adicionar Coluna Quantidade_Por_Tipo

In [95]:
df["Quantidade"] = df.index + 1
df["Quantidade_Por_Tipo"] = ""

In [96]:
for tipo in df["Tipo"].unique():
    lista_indices = df[df["Tipo"]==tipo].index
    for idx, row in enumerate(lista_indices, start = 1):
        df.loc[row,'Quantidade_Por_Tipo'] = str(idx).zfill(2) + "/" + str(df.loc[row,'Quantidade']).zfill(2)
    

## Preparar DataFrames

In [97]:
total_linhas = df["Linha"].max()
total_tokens = df["Quantidade"].max()

In [98]:
## Isolando colunas desejadas

df = df[['Tipo','Palavra','Linha','Quantidade_Por_Tipo']]

In [99]:
## Renomeando colunas

df = df.rename(columns = {
    'Tipo' : 'Classe'
    , 'Palavra' : 'token'
    , 'Linha' : 'linha'
    , 'Quantidade_Por_Tipo' : 'quantidade (1ª coluna(qtdade da classe), 2ª qtdade de tokens)'
    })

In [100]:
df

,Classe,token,linha,"quantidade (1ª coluna(qtdade da classe), 2ª qtdade de tokens)"
0,palavra reservada,begin,1,01/01
1,palavra reservada,end,1,02/02
2,palavra reservada,if,1,03/03
3,pontuação,;,1,01/04
4,pontuação,:,1,02/05
5,operadores,:=,1,01/06
6,palavra reservada,while,2,04/07
7,palavra reservada,for,2,05/08
8,palavra reservada,then,2,06/09
9,inteiro,20,2,01/10


In [101]:
df_agrupado = df.groupby(by = 'Classe').size().reset_index(name='quantidade')

In [102]:
df_agrupado.loc[len(df_agrupado)] = ["Total linhas", total_linhas]
df_agrupado.loc[len(df_agrupado)] = ["Total tokens", total_tokens]

In [103]:
df_agrupado["quantidade"] = df_agrupado["quantidade"].astype(str).str.zfill(2)

In [104]:
df_agrupado.loc[df_agrupado["Classe"]=="inteiro","Classe"] = "números inteiros"
df_agrupado.loc[df_agrupado["Classe"]=="real","Classe"] = "números reais"
df_agrupado.loc[df_agrupado["Classe"]=="variável","Classe"] = "variáveis"

In [105]:
df_agrupado

,Classe,quantidade
0,números inteiros,01
1,operadores,01
2,palavra reservada,07
3,pontuação,02
4,números reais,01
5,variáveis,02
6,Total linhas,02
7,Total tokens,14


## Arquivo de Saída

In [106]:
nome_arquivo_saida = f'output/{nome_arquivo.split(".p")[0]}.csv'
df.to_csv(nome_arquivo_saida, index = False)

In [107]:
nome_arquivo_saida = f'output/{nome_arquivo.split(".p")[0]}_stats.csv'
df_agrupado.to_csv(nome_arquivo_saida, index = False)

In [108]:
print(df.to_markdown(index = False, tablefmt="fancy_grid"))

╒═══════════════════╤═════════╤═════════╤═════════════════════════════════════════════════════════════════╕
│ Classe            │ token   │   linha │ quantidade (1ª coluna(qtdade da classe), 2ª qtdade de tokens)   │
╞═══════════════════╪═════════╪═════════╪═════════════════════════════════════════════════════════════════╡
│ palavra reservada │ begin   │       1 │ 01/01                                                           │
├───────────────────┼─────────┼─────────┼─────────────────────────────────────────────────────────────────┤
│ palavra reservada │ end     │       1 │ 02/02                                                           │
├───────────────────┼─────────┼─────────┼─────────────────────────────────────────────────────────────────┤
│ palavra reservada │ if      │       1 │ 03/03                                                           │
├───────────────────┼─────────┼─────────┼─────────────────────────────────────────────────────────────────┤
│ pontuação         │ ;     

In [109]:
print(df_agrupado.to_markdown(index = False, tablefmt="fancy_grid"))

╒═══════════════════╤══════════════╕
│ Classe            │   quantidade │
╞═══════════════════╪══════════════╡
│ números inteiros  │           01 │
├───────────────────┼──────────────┤
│ operadores        │           01 │
├───────────────────┼──────────────┤
│ palavra reservada │           07 │
├───────────────────┼──────────────┤
│ pontuação         │           02 │
├───────────────────┼──────────────┤
│ números reais     │           01 │
├───────────────────┼──────────────┤
│ variáveis         │           02 │
├───────────────────┼──────────────┤
│ Total linhas      │           02 │
├───────────────────┼──────────────┤
│ Total tokens      │           14 │
╘═══════════════════╧══════════════╛
